In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os,cv2,re,random
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array , load_img
from keras import layers,models,optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split
    
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['train', 'test']


In [2]:

train_dir = "../input/train/"
test_dir = "../input/test/"
train_img_dogs_cat = [train_dir + i for i in os.listdir(train_dir)]#use this f or use whole dataset
test_img_dogs_cat = [test_dir + i for i in os.listdir(test_dir)]


Helper function to sort the image files based on the numeric value in each file name.

In [3]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text) ]

Sort the traning set. Use 1300 images each of cats and dogs instead of all 25000 to speed up the learning process.

Sort the test set

In [4]:
train_img_dogs_cat.sort(key = natural_keys)
train_img_dogs_cat = train_img_dogs_cat[0:1300] + train_img_dogs_cat[12500 :13800]
test_img_dogs_cat.sort(key = natural_keys)


Now the images have to be represented in numbers. For this, using the openCV library read and resize the image.

Generate labels for the supervised learning set.

Below is the helper function to do so.

In [5]:
img_height = 150
img_width = 150


def prepare_data(list_of_images):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """
    x = [] # images as arrays
    y = [] # labels
    
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (img_width,img_height), interpolation=cv2.INTER_CUBIC))
    
    for i in list_of_images:
        if 'dog' in i:
            y.append(1)
        elif 'cat' in i:
            y.append(0)
        #else:
            #print('neither cat nor dog name present in images')
            
    return x, y
    
    

Generate X and Y using the helper function above

Since K.image_data_format() is channel_last, input_shape to the first keras layer will be (img_width, img_height, 3). '3' since it is a color image

In [6]:
X,Y = prepare_data(train_img_dogs_cat)
print(K.image_data_format())

channels_last


Split the data set containing 2600 images into 2 parts, training set and validation set. Later, you will see that accuracy and loss on the validation set will also be reported while fitting the model using training set.****

In [7]:
X_train, X_val ,y_train, y_val = train_test_split(X,Y,test_size = 0.2 ,random_state = 1)

In [8]:
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
batch_size = 16

We will be using the Sequential model from Keras to form the Neural Network. Sequential Model is used to construct simple models with linear stack of layers.

More info on Sequential model and Keras in general at https://keras.io/getting-started/sequential-model-guide/ and https://github.com/keras-team/keras

In [9]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),input_shape=(img_width,img_height,3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size = (2,2)))

model.add(layers.Conv2D(32,(3,3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size = (2,2)))

model.add(layers.Conv2D(64,(3,3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size = (2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
__________

This is the augmentation configuration we will use for training and validation



In [10]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

Prepare generators for training and validation sets

In [11]:
train_generator = train_datagen.flow(np.array(X_train),y_train,batch_size = batch_size)
validation_generator = train_datagen.flow(np.array(X_val),y_val,batch_size = batch_size)

Start training the model!

For better accuracy and lower loss, we are using an epoch of 30. Epoch value can be increased for better results.

In [12]:
hystory = model.fit_generator(
                train_generator,
                steps_per_epoch = nb_train_samples // batch_size,
                epochs = 30,
                validation_data = validation_generator,
                validation_steps=nb_validation_samples // batch_size
                 )

Epoch 1/30
130/130 [==============================] - 42s 321ms/step - loss: 0.7486 - acc: 0.5183 - val_loss: 0.6871 - val_acc: 0.5176
Epoch 2/30
130/130 [==============================] - 41s 317ms/step - loss: 0.6872 - acc: 0.5596 - val_loss: 0.6486 - val_acc: 0.6190
Epoch 3/30
130/130 [==============================] - 41s 316ms/step - loss: 0.6680 - acc: 0.6149 - val_loss: 0.6488 - val_acc: 0.6409
Epoch 4/30
130/130 [==============================] - 41s 318ms/step - loss: 0.6509 - acc: 0.6385 - val_loss: 0.8240 - val_acc: 0.5119
Epoch 5/30
130/130 [==============================] - 41s 318ms/step - loss: 0.6544 - acc: 0.6452 - val_loss: 0.6830 - val_acc: 0.5972
Epoch 6/30
130/130 [==============================] - 42s 321ms/step - loss: 0.6319 - acc: 0.6760 - val_loss: 0.6168 - val_acc: 0.6806
Epoch 7/30
130/130 [==============================] - 41s 317ms/step - loss: 0.6237 - acc: 0.6745 - val_loss: 0.6106 - val_acc: 0.6766
Epoch 8/30
130/130 [==============================] - 4

In [13]:
X_test, Y_test = prepare_data(test_img_dogs_cat) #Y_test in this case will be []

In [14]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [15]:
steps = nb_train_samples // batch_size
test_generator = val_datagen.flow(np.array(X_test), batch_size=batch_size)

prediction_probabilities = model.predict_generator(test_generator, verbose=1,steps=782)

782/782 [==============================] - 86s 110ms/step


In [16]:
counter = range(1, len(test_img_dogs_cat) + 1)
solution = pd.DataFrame({"id": counter, "label":list(prediction_probabilities)})
cols = ['label']

for col in cols:
    solution[col] = solution[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)

solution.to_csv("dogsVScats.csv", index = False)